In [4]:
pip install selenium

  Using cached selenium-4.19.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached trio-0.25.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.19.0-py3-none-any.whl (10.5 MB)
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ---------------------- ----------------- 92.2/163.8 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 163.8/163.8 kB 3.3 MB/s eta 0:00:00
Using cached trio-0.25.0-py3-none-any.whl (467 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
   ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
   ---------------------------------------- 121.1/121.1 kB 6.9 MB/s et

In [5]:
pip install webdriver-manager

  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

from webdriver_manager.chrome import ChromeDriverManager

def find_element_or_none(doc, element: str, by_selector= True):
    try:
        return doc.find_element(By.CSS_SELECTOR if by_selector else By.ID, element).text
    except (NoSuchElementException):
        return None
    except (StaleElementReferenceException):
        time.sleep(1)
        return doc.find_element(By.CSS_SELECTOR if by_selector else By.ID, element).text

def scrape_page(driver):            
    # Selector list of offers
    job_list_sel = '#jsJobResContent .jobItem'
    
    # Selectors fields for each item in the list of offers
    job_item_title_sel = '.jobResultsTitle'
    job_item_salary_sel = '.jobResultsSalary'
    job_item_loc_sel = '.jobResultsLoc'
    job_item_type_sel = '.jobResultsType'
    job_item_when_sel = '.when'

    # Job details description
    job_detail_skills_id = 'md_skills'

    # Job details fields after the description
    # job_detail_location_id = '#md_location'
    # job_detail_industry_id = '#md_industry'
    # job_detail_category_id = '#md_category'
    job_detail_duration_id = 'md_duration'
    job_detail_start_date_id = 'md_start_date'
    #job_detail_rate_sel= '#md_rate'
    job_detail_recruiter_id = 'md_recruiter'
    job_detail_ref_id = 'md_ref'
    job_detail_posted_date_id = 'md_posted_date'
    job_detail_permalink_id = 'md_permalink'

    # Find all job offer items in the list
    job_items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, job_list_sel)))
    print(f"Job_tems found: {len(job_items)}")
    
    for index, job_item in enumerate(job_items):
        print(f"**Job Index: {index + 1}")

        # Click on the job item to load its details into the div; text outside the view can't be seen by Selenium
        ActionChains(driver).move_to_element(job_item).click(job_item).perform()
        time.sleep(1)

        # Extract details from current job item
        title = find_element_or_none(job_item, job_item_title_sel)
        salary = find_element_or_none(job_item, job_item_salary_sel)
        loc = find_element_or_none(job_item, job_item_loc_sel)
        type = find_element_or_none(job_item, job_item_type_sel)
        when = find_element_or_none(job_item, job_item_when_sel)
        skills = ""
        duration = None
        start_date = None
        recruiter = None
        ref = None
        posted_date = None
        permalink = None

        # # Simulate a page reload or dynamic content update
        # driver.refresh()

        # Select the window slider
        div_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".jsCustomScrollContainer")))
        time.sleep(1)

        # Get the scrollable height of the div element
        scroll_height = driver.execute_script("return arguments[0].scrollHeight;", div_element)
        print(f"scroll_height: {scroll_height}")

        current_scroll_position = 1
        increase = 100
        while current_scroll_position < scroll_height:
            driver.execute_script(f"arguments[0].style.top = '-{current_scroll_position}px';", div_element)
            current_scroll_position += increase
            # Wait for some time to allow content to load (you may adjust the time as needed)
            time.sleep(1)
            
            skills_try = find_element_or_none(div_element, job_detail_skills_id, False)
            skills += skills_try if skills_try else ""
            duration_try = find_element_or_none(div_element, job_detail_duration_id, False)
            duration = duration_try if duration_try else duration
            start_date_try = find_element_or_none(div_element, job_detail_start_date_id, False)
            start_date = start_date_try if start_date_try else start_date
            #rate = find_element_or_none(job_detail_container, job_detail_rate_id)
            recruiter_try = find_element_or_none(div_element, job_detail_recruiter_id, False)
            recruiter = recruiter_try if recruiter_try else recruiter
            ref_try = find_element_or_none(div_element, job_detail_ref_id, False)
            ref = ref_try if ref_try else ref
            posted_date_try = find_element_or_none(div_element, job_detail_posted_date_id, False)
            posted_date = posted_date_try if posted_date_try else posted_date
            permalink_try = find_element_or_none(div_element, job_detail_permalink_id, False)
            permalink = permalink_try if permalink_try else permalink

        # field_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, f"//div[@id='{job_detail_permalink_sel}']//*")))
        
        # ActionChains(driver).move_to_element(element).perform()

        print(f"({index + 1})*Item\ntitle:{title}\nsalary:{salary}\nloc:{loc}\ntype:{type}\nwhen:{when}\n")
        print(f"({index + 1})*Details\nskills:{skills}\nduration:{duration}\nstart_date:{start_date}\nrecruiter:{recruiter}\nref:{ref}\nposted_date:{posted_date}\npermalink:{permalink}\n")


def scrape_job_offers(url):
    # Setup Chrome options
    chrome_options = Options()
    # Runs Chrome in headless mode.
    # chrome_options.add_argument("--headless")

    # Setup Selenium with ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Navigate to the page
    driver.get(url)

    try:
        scrape_page(driver)
    finally:        
        time.sleep(10)
        # Close the browser
        driver.quit()

if __name__ == "__main__":
    ## Edit Search
    # Keywords: python "machine learning" engineer
    # Industry: All
    # Job title: python engineer
    # Location: London (Only show job with remote working)
    # Category: none
    # Salary: All
    # Within: 7days
    # Job type: Any
    # Advertiser: Any
    # Distance: 5miles
    url = "https://www.jobserve.com/gb/en/JobSearch.aspx?shid=2981D9787360CBB0C8BE"

    scrape_job_offers(url)
    print("Ended")


Job_tems found: 25
**Job Index: 1
scroll_height: 967
(1)*Item
title:Python Engineer - Front Office
salary:£120k - £130k per annum
loc:London
type:Permanent
when:5 days ago

(1)*Details
skills:Python Engineer - Front Office
Location: London
Working Arrangements: Hybrid (2-3 days p/w in office)
Salary: £120,000 - £130,000
Industry: Consultancy/Finance
Tech Stack: Python, SQL, Pandas, Numpy
Great opportunity for a talented Developer (Python, SQL, Pandas, Numpy) to join a tech consultancy that deliver greenfield platforms for their clients.
The Company
Renowned boutique consultancy working with one of the largest investment banks in the world, they operate across the globe and a renowned household name.
The RoleWorking Arrangements: Hybrid (2-3 days p/w in office)
Salary: £120,000 - £130,000
Industry: Consultancy/Finance
Tech Stack: Python, SQL, Pandas, Numpy
Great opportunity for a talented Developer (Python, SQL, Pandas, Numpy) to join a tech consultancy that deliver greenfield platforms